In [11]:
# import the packages
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from matplotlib import pyplot as plt
import numpy as np

# import Arizona cps 2021 data (tab.1) and automation risk data (tab.2)
df1=pd.read_csv("/Users/luyudu/Desktop/Data/cpsaz_selected_2021.csv")
df2=pd.read_csv("/Users/luyudu/Desktop/Data/automation_risk_occ.csv")

# define automation risks
def find_risk_lvl(row):
    if row.ComputerisableProbability>0.7:
        return "High: P>0.7"
    if (row.ComputerisableProbability<=0.7)&(row.ComputerisableProbability>=0.3):
        return "Medium: P [0.3-0.7]"
    if row.ComputerisableProbability<0.3:
        return "Low: P<0.3"   
df2['risk_level']=df2.apply(find_risk_lvl,axis=1)

# define age group
def find_age(row):
    if ((float(row.age)>=18) and (float(row.age)<=35)):
        return "Young: 18-35"
    if ((float(row.age)>=36) and (float(row.age)<=55)):
        return "Middle-Age: 36-55"
    if (float(row.age)>=56):
        return "Old: >56"
df1['age2']=df1.apply(find_age,axis=1)
df1.head(5)

# import labor statistics data (tab.3), which contains major occupational group codes OCC_CODE
df3=pd.read_csv("/Users/luyudu/Desktop/Data/LBS2021_az_selected.csv")

def find_wage(row):
    if (pd.notnull(row.ann_median_wage) and (float(row.ann_median_wage)<=30000)):
        return "<30K"
    if (pd.notnull(row.ann_median_wage) and (float(row.ann_median_wage)>30000) and (float(row.ann_median_wage)<=60000)):
        return "30K-60K"
    if (pd.notnull(row.ann_median_wage) and (float(row.ann_median_wage)>60000) and (float(row.ann_median_wage)<=100000)):
        return "60K-100K"
    if (pd.notnull(row.ann_median_wage) and (float(row.ann_median_wage)>100000)):
        return ">100K"
df3['wage2']=df3.apply(find_wage, axis=1)


In [12]:
# classify major occupational groups <-- merge tab.2 and tab.3
# split OCC_CODE (eg.53-000, where "53" stands for the transportation industry and "000" indicates specific jobs in this industry
df3_risk=df3.merge(df2,how='inner',on='Occupation')
df3_risk['CODE0']=df3_risk['OCC_CODE'].str.split('-',1).str[0]
df3_risk['CODE0'] = df3_risk['CODE0'].astype(int)
df3_risk.head(5)

# merge all three datasets: tab.1,tab.2, and tab.3
df1_risk=df1.merge(df3_risk,how='inner',on='Occupation')
df1_risk['ComputerisableProbability'] = df1_risk['ComputerisableProbability'].astype(float)
df1_risk.head(5)

### Copy the merged table of tab2 and tab3. Find job number percentage in job risk
df4_risk=df3_risk

# Drop empty rows
df4_risk['TOT_EMP'].replace('',np.nan, inplace=True)
df4_risk.dropna(subset=['TOT_EMP'], inplace=True)

# Since TOT_EMP is object type, which cannot be performed by math, data conversion is needed
df4_risk['TOT_EMP2'] = df4_risk['TOT_EMP'].astype(int)

# do a sum based on the same risk level
df4_risk_job=df4_risk.groupby(['risk_level']).sum().reset_index()
SUM_TOT_EMP2=sum(df4_risk_job.TOT_EMP2)
df4_risk_job['percent']=(100*df4_risk_job.TOT_EMP2/SUM_TOT_EMP2).round(0)
df4_risk_job.head(3)

# percentage plot of risk level for AZ transportation jobs only
df4_risk_53=df4_risk[df4_risk['CODE0']==53]
counts_risk_53=df4_risk_53.groupby(['risk_level']).size().reset_index(name='counts').sort_values('risk_level')
total_counts_53=sum(counts_risk_53.counts)
counts_risk_53['percent']=(100*counts_risk_53.counts/total_counts_53).round(0)

# percentage plot of risk level for all AZ jobs, and combine into the same graph
df4_risk_job['All Jobs']=df4_risk_job['percent']
counts_risk_53['Transportation Jobs']=counts_risk_53['percent']
df4_risk_job_merge=df4_risk_job.merge(counts_risk_53,how='inner',on='risk_level')

fig0d = px.bar(df4_risk_job_merge, x="risk_level",y=["All Jobs","Transportation Jobs"], barmode='group',
              title='Figure 1. Automation Risk Levels of All AZ Jobs VS. Transportation Jobs',
              text_auto=True,
              category_orders={"risk_level":["High: P>0.7","Medium: P [0.3-0.7]","Low: P<0.3"]},
              )
fig0d.update_layout(xaxis_title='Automation Risk Level', yaxis_title='Percentage (%)', title_x=0.5)
fig0d.show()


In [13]:
# computerisable probability of AZ transportation jobs

df3_risk_53=df3_risk[df3_risk['CODE0']==53]
fig10=px.bar(df3_risk_53.sort_values('ComputerisableProbability', ascending=True),x='ComputerisableProbability',y='Occupation', text_auto=True)
fig10.update_layout(title_text='Figure 2. Computerisable Probability of Detailed Transportation Occupations in Arizona',title_x=0.5,
                   xaxis_title='Computerisable Probability', yaxis_title='Detailed Transportation Occupations')
fig10.show()



In [14]:
# find demographic data with transportation workers VS all AZ workers and plot bar chart in terms of education level
df1_risk_53=df1_risk[df1_risk['CODE0']==53]

df1_risk_educ=df1_risk.groupby(['educ']).size().reset_index(name='count')
df1_risk_educ['count']=df1_risk_educ['count'].astype(int)
total_count=df1_risk_educ['count'].sum()
df1_risk_educ['All Workers']=(100*df1_risk_educ['count']/total_count).round(0)

df1_risk_53_educ=df1_risk_53.groupby(['educ']).size().reset_index(name='count2')
df1_risk_53_educ['count2']=df1_risk_53_educ['count2'].astype(int)
total_count2=df1_risk_53_educ['count2'].sum()
df1_risk_53_educ['Transportation Workers']=(100*df1_risk_53_educ['count2']/total_count2).round(0)

df1_risk_educ_merge=df1_risk_educ.merge(df1_risk_53_educ,how='inner',on='educ')

fig5c = px.bar(df1_risk_educ_merge, x="educ",y=["All Workers","Transportation Workers"], barmode='group',
              title='Figure 3. Educational Attainment of All AZ Workers VS. Transportation Workers',
              text_auto=True, 
              category_orders={"educ":["Advanced","College","Some college","High school","Less than high school"]})
fig5c.update_layout(xaxis_title='Educational Attainment', yaxis_title='Percentage (%)', title_x=0.5)
fig5c.show()

# find demographic data with transportation workers VS. all AZ workers and plot the bar chart in terms of sex
df1_risk_53=df1_risk[df1_risk['CODE0']==53]

df1_risk_sex=df1_risk.groupby(['sex']).size().reset_index(name='count')
df1_risk_sex['count']=df1_risk_sex['count'].astype(int)
total_count=df1_risk_sex['count'].sum()
df1_risk_sex['All Workers']=(100*df1_risk_sex['count']/total_count).round(0)

df1_risk_53_sex=df1_risk_53.groupby(['sex']).size().reset_index(name='count2')
df1_risk_53_sex['count2']=df1_risk_53_sex['count2'].astype(int)
total_count2=df1_risk_53_sex['count2'].sum()
df1_risk_53_sex['Transportation Workers']=(100*df1_risk_53_sex['count2']/total_count2).round(2)

df1_risk_sex_merge=df1_risk_sex.merge(df1_risk_53_sex,how='inner',on='sex')

fig7c = px.bar(df1_risk_sex_merge, x="sex",y=["All Workers","Transportation Workers"],barmode='group',
              title='Figure 4. Sex of All AZ Workers VS. Transportation Workers',
              text_auto=True, 
              category_orders={"educ":["Male","Female"]})
fig7c.update_layout(xaxis_title='Sex', yaxis_title='Percentage (%)', title_x=0.5)
fig7c.show()

# find demographic data with transportation workers VS. all AZ workers and plot the bar chart in terms of race and ethnicity
df1_risk_53=df1_risk[df1_risk['CODE0']==53]

df1_risk_race=df1_risk.groupby(['race_ethnicity']).size().reset_index(name='count')
df1_risk_race['count']=df1_risk_race['count'].astype(int)
total_count=df1_risk_race['count'].sum()
df1_risk_race['All Workers']=(100*df1_risk_race['count']/total_count).round(0)

df1_risk_53_race=df1_risk_53.groupby(['race_ethnicity']).size().reset_index(name='count2')
df1_risk_53_race['count2']=df1_risk_53_race['count2'].astype(int)
total_count2=df1_risk_53_race['count2'].sum()
df1_risk_53_race['Transportation Workers']=(100*df1_risk_53_race['count2']/total_count2).round(0)

df1_risk_race_merge=df1_risk_race.merge(df1_risk_53_race,how='inner',on='race_ethnicity')

fig8c = px.bar(df1_risk_race_merge, x="race_ethnicity",y=["All Workers","Transportation Workers"],barmode='group',
              title='Figure 5. Race of All AZ Workers VS. Transportation Workers',
              text_auto=True, 
              category_orders={"race_ethnicity":["White","Black","Hispanic","Asian","Other"]})
fig8c.update_layout(xaxis_title='Race and Ethnicity', yaxis_title='Percentage (%)', title_x=0.5)
fig8c.show()

# find demographic data with transportation workers VS. all AZ workers and plot the bar chart in terms of age
df1_risk_53=df1_risk[df1_risk['CODE0']==53]

df1_risk_age2=df1_risk.groupby(['age2']).size().reset_index(name='count')
df1_risk_age2['count']=df1_risk_age2['count'].astype(int)
total_count=df1_risk_age2['count'].sum()
df1_risk_age2['All Workers']=(100*df1_risk_age2['count']/total_count).round(0)

df1_risk_53_age2=df1_risk_53.groupby(['age2']).size().reset_index(name='count2')
df1_risk_53_age2['count2']=df1_risk_53_age2['count2'].astype(int)
total_count2=df1_risk_53_age2['count2'].sum()
df1_risk_53_age2['Transportation Workers']=(100*df1_risk_53_age2['count2']/total_count2).round(0)

df1_risk_age2_merge=df1_risk_age2.merge(df1_risk_53_age2,how='inner',on='age2')

fig9c = px.bar(df1_risk_age2_merge, x="age2",y=["All Workers","Transportation Workers"],barmode='group',
              title='Figure 6. Age of All AZ Workers VS. Transportation Workers',
              text_auto=True, 
              category_orders={"age2":["Young: 18-35","Middle-Age: 36-55","Old: >56"]})
fig9c.update_layout(xaxis_title='Age', yaxis_title='Percentage (%)', title_x=0.5)
fig9c.show()


# find demographic data with transportation workers VS. all AZ workers and plot the bar chart in terms of annual median wage
df1_risk_53=df1_risk[df1_risk['CODE0']==53]

df1_risk_wage2=df1_risk.groupby(['wage2']).size().reset_index(name='count')
df1_risk_wage2['count']=df1_risk_wage2['count'].astype(int)
total_count=df1_risk_wage2['count'].sum()
df1_risk_wage2['All Workers']=(100*df1_risk_wage2['count']/total_count).round(0)

df1_risk_53_wage2=df1_risk_53.groupby(['wage2']).size().reset_index(name='count2')
df1_risk_53_wage2['count2']=df1_risk_53_wage2['count2'].astype(int)
total_count2=df1_risk_53_wage2['count2'].sum()
df1_risk_53_wage2['Transportation Workers']=(100*df1_risk_53_wage2['count2']/total_count2).round(0)

df1_risk_wage2_merge=df1_risk_wage2.merge(df1_risk_53_wage2,how='inner',on='wage2')

fig11c = px.bar(df1_risk_wage2_merge, x="wage2",y=["All Workers","Transportation Workers"],barmode='group',
              title='Figure 7. Annual Median Wage of All AZ Workers VS. Transportation Workers',
              text_auto=True, 
              category_orders={"wage2":["<30K","30K-60K","60K-100K"]})
fig11c.update_layout(xaxis_title='Annual Median Wage', yaxis_title='Percentage (%)', title_x=0.5)
fig11c.show()
